<a href="https://colab.research.google.com/github/undeadcycle/Capstone/blob/main/Folium(0).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [112]:
!pip3 install folium
!pip3 install wget

In [113]:
import folium
import wget
import pandas as pd
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

In [114]:
spacex_csv_file = wget.download('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv')
spacex_df=pd.read_csv(spacex_csv_file)

In [115]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610746


In [116]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)
for index, row in launch_sites_df.iterrows():
  circle = folium.CircleMarker(
        location=[row["Lat"], row["Long"]],
        # radius=1000, 
        color='#d35400', 
        fill=True,
        ).add_child(folium.Popup(row['Launch Site']))
  site_map.add_child(circle)

In [117]:
site_map

In [118]:
# spacex_df.tail(10)

In [119]:
def assign_marker_color(launch_outcome):
    if launch_outcome == 1:
        return 'green'
    else:
        return 'red'
    
spacex_df['marker_color'] = spacex_df['class'].apply(assign_marker_color)
# spacex_df.tail(10)

In [120]:
spacex_df.rename({'class': 'Class', 'Launch Site':'Launch_Site', 'marker_color':'Marker_Color'}, axis=1, inplace=True)

In [121]:
# spacex_df.dtypes

In [122]:
%%capture
site_map.add_child(marker_cluster)

In [123]:
marker_cluster = MarkerCluster().add_to(site_map)

for row in spacex_df.itertuples():
    folium.Marker(location=[row.Lat,row.Long],
                  popup=row.Launch_Site, 
                  icon=folium.Icon(color='black', icon_color=row.Marker_Color)).add_to(marker_cluster)

In [124]:
# %%capture
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)

In [125]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [126]:
coastline_lat = 28.56274
coastline_lon = -80.56764
launch_site_lat = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'KSC LC-39A']['Lat'].values[0]
launch_site_lon = launch_sites_df.loc[launch_sites_df['Launch Site'] == 'KSC LC-39A']['Long'].values[0]
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline

7.8300965695680596

In [127]:
%%capture
coordinate = [28.56274, -80.56764]
distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_marker = folium.Marker(
   coordinate,
   icon=DivIcon(
       icon_size=(20,20),
       icon_anchor=(0,0),
       html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % "{:10.2f} KM".format(distance),
       )
   )
site_map.add_child(distance_marker)

In [128]:
lc40 = []
lc40.append(launch_sites_df.iloc[0]['Lat'])
lc40.append(launch_sites_df.iloc[0]['Long'])
# lc40

In [129]:

line = [coordinate,lc40]
# line

In [130]:
# %%capture
lines=folium.PolyLine(locations=line, weight=1)
site_map.add_child(lines)